In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

fpath = '../example/data/data_processed_1prod_full.json'
df = pd.read_json(fpath, lines=True)
docs = df['reviewText'].astype(str)

In [2]:
from orangecontrib.argument.miner.topic import ArguTopic, chunker

docs = chunker(docs)
docs.head()

/Volumes/Databank/Workspace/orange3-argument/venv/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Volumes/Databank/Workspace/orange3-argument/venv/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Volumes/Databank/Workspace/orange3-argu

,doc_id,chunk
0,0,I always get a half size up in my tennis shoes .
1,0,For some reason these feel to big in the heel ...
2,1,walked 3 hours with no problem
3,1,Put them on and !
4,1,Love them !


In [3]:
topic_model = ArguTopic()

PartOfSpeech(model=<spacy.lang.en.English object at 0x29f4caf50>,
             pos_patterns=[[{'POS': 'NOUN'}], [{'POS': 'ADJ'}],
                           [{'POS': 'VERB'}]])

In [4]:
topics, probs = topic_model.fit_transform_reduce_outliers(docs['chunk'])

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,28,-1_excelente_stylish_footbed_intended
1,0,88,0_favorite_amazing_turned_liked
2,1,87,1_fits_glove_fit_perfect
3,2,78,2_half_ordered_order_size
4,3,61,3_attractive_cushion_bad_allow
5,4,40,4_purchase_pleased_satisfied_product
6,5,53,5_weight_light_lightweight_minimalist
7,6,48,6_super_mesh_job_comfy
8,7,50,7_class_zumba_concrete_aerobics
9,8,60,8_compliments_airport_received_look


In [ ]:
topic_model.get_topic(23)

[('worn', 0.6487922045950961),
 ('day', 0.6346370270492367),
 ('row', 0.5802676574079804),
 ('wore', 0.556969065871802),
 ('able', 0.5390788818856214),
 ('changed', 0.52149003016946),
 ('days', 0.4815547040615173),
 ('years', 0.4587653674501624),
 ('wearing', 0.4490367764967294),
 ('daily', 0.44015606390995127)]